In [1]:
# For Kaggle training downgrade the keras library
!pip install keras==2.15.0 tensorflow==2.15.0 -q

# Restart session

ERROR: Could not find a version that satisfies the requirement tensorflow==2.15.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.15.0


In [2]:
# required imports
import os
import requests
from zipfile import ZipFile
import tensorflow as tf

In [3]:
def download_file(url, save_name):
  file = requests.get(url)
  open(save_name, 'wb').write(file.content)
  return

In [5]:
def unzip(zip_file_pzth=None):
    try:
        with ZipFile(zip_file_path) as z:
            z.extractall("./")
            print(f"Extracted {zip_file_path}...\n")
    except:
        print("Invalid file")

    return

In [6]:
if not os.path.exists('models'):
    download_file(
                  'https://www.dropbox.com/s/bqedbdn58sw5dl5/detector_nn_architecture.zip?dl=1',
                  'detector_nn_architecture.zip'
                 )

    unzip('detector_nn_architecture.zip')

Invalid file


In [7]:
os.listdir("./")

['.config', 'detector_nn_architecture.zip', 'sample_data']

In [9]:
import os
os.getcwd()

'/content'

In [14]:
!unzip detector_nn_architecture.zip -d .
os.listdir("./")

Archive:  detector_nn_architecture.zip
   creating: ./models/
   creating: ./models/resnet18_notop/
  inflating: ./models/resnet18_notop/saved_model.pb  
  inflating: ./models/resnet18_notop/keras_metadata.pb  
   creating: ./models/resnet18_notop/variables/
  inflating: ./models/resnet18_notop/variables/variables.index  
  inflating: ./models/resnet18_notop/variables/variables.data-00000-of-00001  
   creating: ./models/resnet18_notop/assets/
   creating: ./test_images/
  inflating: ./test_images/FudanPed00002.png  
  inflating: ./test_images/FudanPed00001.png  
  inflating: ./fpn.py                
  inflating: ./detector.py           


['.config',
 'detector.py',
 'test_images',
 'detector_nn_architecture.zip',
 'models',
 'fpn.py',
 'sample_data']

In [15]:
import sys
sys.path.append('/content')

In [16]:
# Detector Architecture
from detector import get_detector

In [21]:
input_shape = (300, 300, 3)
backbone_name = "resnet_18"
num_classes = 2
fpn_channels = 32
num_anchors = 9

In [23]:
detector = get_detector(
    input_shape=input_shape,
    backbone_name="resnet_18",
    num_classes=num_classes,
    fpn_channels=fpn_channels,
    num_anchors=num_anchors
)

tf.keras.utils.plot_model(detector, to_file='detector.png', show_shapes=True, rankdir='TB')

ValueError: File format not supported: filepath=/content/models/resnet18_notop. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(/content/models/resnet18_notop, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [ ]:
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from detector import get_backbone

input_layer = tf.keras.Input(shape=input_shape)
preprocess_modes = {"resnet_18": "torch", "resnet_50": "caffe"}
model_input = preprocess_input(input_layer, mode=preprocess_modes[backbone_name])
backbone = get_backbone(backbone_name=backbone_name)
layer_3_out, layer_4_out, layer_5_out = backbone(model_input)

In [ ]:
from fpn import Lateral_Connection

def get_fpn(backbone_out, channels_out):

    # FPN inputs
    in_backbone_out_3, in_backbone_out_4, in_backbone_out_5 = backbone_out

    # 1x1 conv output
    p5_out = tf.keras.layers.Conv2D(channels_out, 1, 1, "same", name="p5_out_1x1")(in_backbone_out_5)

    # lateral connection
    p4_out = Lateral_Connection(channels_out, name="lat_con_p4_out")((p5_out, in_backbone_out_4))
    p3_out = Lateral_Connection(channels_out, name="lat_con_p3_out")((p4_out, in_backbone_out_3))

    # 3x3 conv output
    p3_out = tf.keras.layers.Conv2D(channels_out, 3, 1, "same", name="p3_out_3x3")(p3_out)
    p4_out = tf.keras.layers.Conv2D(channels_out, 3, 1, "same", name="p4_out_3x3")(p4_out)
    p5_out = tf.keras.layers.Conv2D(channels_out, 3, 1, "same", name="p5_out_3x3")(p5_out)

    fpn = tf.keras.Model(inputs=[in_backbone_out_3, in_backbone_out_4, in_backbone_out_5],
                         outputs=[p3_out, p4_out, p5_out], name="FPN")
    return fpn

In [ ]:
# Get the FPN model
fpn_model = get_fpn(backbone_out=(layer_3_out, layer_4_out, layer_5_out), channels_out=32)
# Plot the FPN model
tf.keras.utils.plot_model(fpn_model, to_file='fpn.png', show_shapes=True, rankdir='TB')

In [ ]:
# Inference

import cv2

img1 = cv2.imread("./test_images/FudanPed00001.png")
img2 = cv2.imread("./test_images/FudanPed00002.png")

img1 = tf.convert_to_tensor(cv2.resize(img1, (300, 300)) / 255., dtype = tf.float32)
img2 = tf.convert_to_tensor(cv2.resize(img2, (300, 300)) / 255., dtype = tf.float32)

images = tf.convert_to_tensor([img1, img2])

# Already initialized the detector model. Now let's predict
preds = detector.predict(images)

print('location_pred size: {}'.format(preds[0].shape))
print('class_pred size: {}'.format(preds[1].shape))